#### 1. pb文件的保存方法。
#### 将tensorflow计算图和变量取值统一保存在一个文件中。

In [2]:
import tensorflow as tf
from tensorflow.python.framework import graph_util # 

v1 = tf.Variable(tf.constant(1.0, shape=[1]), name = "v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name = "v2")
result = v1 + v2

init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    # 导出计算图的GraphDef部分，这一部分就可以完成推理过程。
    graph_def = tf.get_default_graph().as_graph_def()
    # 将图中的变量及其取值转化为常量，并将图中不必要的节点去掉
    # ['add']给出了需要保存的节点的名字。
    output_graph_def = graph_util.convert_variables_to_constants(sess, graph_def, ['add'])
    with tf.gfile.GFile("saved_model/combined_model.pb", "wb") as f: # 写入文件
           f.write(output_graph_def.SerializeToString())

INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.


#### 2. 加载pb文件。

In [ ]:
from tensorflow.python.platform import gfile
with tf.Session() as sess:
    model_filename = "saved_model/combined_model.pb"
   
    with gfile.FastGFile(model_filename, 'rb') as f: # read byte
        graph_def = tf.GraphDef() 
        # Parse GraphDef
        graph_def.ParseFromString(f.read())
    # ["add:0"]: tensor name
    result = tf.import_graph_def(graph_def, return_elements=["add:0"])
    print sess.run(result)